#Identifying Game Review Attribution with Alibi Gradients

Authors: Jason Phillips & Peeja Khorashadi

Date: 05/01/2023

CSC 215-01

Summary: Utilizing a dataset of Steam reviews for popular games, we are trying to identify the key attributes/words that contribute to a person recommending the game or not.

Dataset:https://www.kaggle.com/datasets/arashnic/game-review-dataset

Papers Refrenced: https://ieeexplore-ieee-org.proxy.lib.csus.edu/document/9752380 , https://ieeexplore-ieee-org.proxy.lib.csus.edu/document/9950194 , https://www.kaggle.com/code/mehmetlaudatekman/game-review-classification-deep-learning-based , https://github.com/SeldonIO/alibi

#Import and Define
Import necessary libraries and define useful functions

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install tf-explain
!pip install --upgrade gensim
!pip install alibi[tensorflow]
#%pip install --upgrade gensim
#%pip install --upgrade numpy

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.2/524.2 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 111.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.4 MB/s eta 0:00:00


In [6]:
#Import and print out the current version of packages.
%load_ext tensorboard

import tensorflow as tf
#import tf_explain
import numpy as np
import matplotlib.pyplot as plt
import sys
import sklearn as sk
import pandas as pd
import shutil
import os
import imblearn
import io
import requests
import datetime
import string

from alibi.explainers import IntegratedGradients

from scipy.stats import zscore
from scipy import spatial

from collections.abc import Sequence
from collections import Counter
from numpy import where
from matplotlib import pyplot

from imblearn.over_sampling import SMOTE

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D,LSTM
from keras.activations import relu, sigmoid, tanh
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import optimizers

from gensim.models import Word2Vec, KeyedVectors
from nltk.stem.lancaster import LancasterStemmer
from nltk.tokenize import RegexpTokenizer
from pprint import pprint

# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column. 
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) * (normalized_high - normalized_low) + normalized_low

# Plot a confusion matrix.
# cm is the confusion matrix, names are the names of the classes.
def plot_confusion_matrix(cm, names, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(names))
    plt.xticks(tick_marks, names, rotation=45)
    plt.yticks(tick_marks, names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

# Plot an ROC. pred - the predictions, y - the expected output.
def plot_roc(pred,y):
    fpr, tpr, thresholds = roc_curve(y, pred)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.show()

def to_sequences(seq_size, data):
    x = []
    y = []

    for i in range(len(data)-SEQUENCE_SIZE-1):
        #print(i)
        window = data[i:(i+SEQUENCE_SIZE)]
        after_window = data[i+SEQUENCE_SIZE]
        window = [[x] for x in window]
        #print("{} - {}".format(window,after_window))
        x.append(window)
        y.append(after_window)
        
    return np.array(x),np.array(y)

print('Python {}'.format(sys.version))
print('Pandas {}'.format(pd.__version__))
print('Numpy {}'.format(np.__version__))
print('Scikit-Learn {}'.format(sk.__version__))
print()
print('Tensor Flow {}'.format(tf.__version__))
print('Keras {}'.format(tf.keras.__version__))
print('Imblearn {}'.format(imblearn.__version__))

#Remove any files in the log folder for tensorboard
#!rm -rf "/content/drive/MyDrive/Colab Notebooks/logs/"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Python 3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]
Pandas 1.5.3
Numpy 1.22.4
Scikit-Learn 1.2.2

Tensor Flow 2.12.0
Keras 2.12.0
Imblearn 0.10.1


In [3]:
#Define the log folder for tensorboard
log_dir = "/content/drive/MyDrive/Colab Notebooks/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

#Data Preprocessing (Training Dataset)
NOTES: 

**Should we include numbers in the tokenized text?**

**Will different genres effect our model?** 

In [7]:
#Read the review dataset from the training folder
filename_read = "https://raw.githubusercontent.com/JasonTPhillipsJr/CSC215/main/Final%20Project/train.csv"
df_train = pd.read_csv(filename_read, sep=',', header=0)
df_train

,review_id,title,year,user_review,user_suggestion
0,1,Spooky's Jump Scare Mansion,2016.0,I'm scared and hearing creepy voices. So I'll...,1
1,2,Spooky's Jump Scare Mansion,2016.0,"Best game, more better than Sam Pepper's YouTu...",1
2,3,Spooky's Jump Scare Mansion,2016.0,"A littly iffy on the controls, but once you kn...",1
3,4,Spooky's Jump Scare Mansion,2015.0,"Great game, fun and colorful and all that.A si...",1
4,5,Spooky's Jump Scare Mansion,2015.0,Not many games have the cute tag right next to...,1
...,...,...,...,...,...
17489,25535,EverQuest II,2012.0,Arguably the single greatest mmorp that exists...,1
17490,25536,EverQuest II,2017.0,"An older game, to be sure, but has its own cha...",1
17491,25537,EverQuest II,2011.0,When I frist started playing Everquest 2 it wa...,1
17492,25538,EverQuest II,NaN,cool game. THe only thing that REALLY PISSES M...,1


In [8]:
#Print out the different games used in this review.
#NOTE: Remove the ® symbol from smite and elder scrolls later.
print("Number of titles in the dataset: " + str(df_train['title'].nunique()))
print(df_train['title'].unique())

Number of titles in the dataset: 44
["Spooky's Jump Scare Mansion" 'Sakura Clicker' 'WARMODE'
 'Fractured Space' 'Dota 2' 'Path of Exile' 'War Thunder'
 'Ring of Elysium' 'Team Fortress 2' 'Yu-Gi-Oh! Duel Links' 'SMITE®'
 'Brawlhalla' 'World of Tanks Blitz' 'DCS World Steam Edition'
 'Heroes & Generals' 'The Elder Scrolls®: Legends™' 'Trove' 'Neverwinter'
 'Realm Royale' 'PlanetSide 2' 'Realm of the Mad God' 'Elsword'
 'theHunter Classic' 'Eternal Card Game' 'Black Squad'
 'Freestyle 2: Street Basketball' 'Bless Online'
 'RaceRoom Racing Experience' 'Fallout Shelter'
 'Tactical Monsters Rumble Arena' 'Creativerse' 'Dreadnought'
 'Infestation: The New Z' 'Shop Heroes' 'Robocraft'
 'Business Tour - Board Game with Online Multiplayer'
 'Crusaders of the Lost Idols' 'AdventureQuest 3D' 'Realm Grinder'
 'World of Guns: Gun Disassembly' 'Bloons TD Battles' 'Cuisine Royale'
 'School of Dragons' 'EverQuest II']


In [9]:
#Drop unnecessary columns
df_train = df_train.drop(['review_id', 'year'], axis=1)
df_train

,title,user_review,user_suggestion
0,Spooky's Jump Scare Mansion,I'm scared and hearing creepy voices. So I'll...,1
1,Spooky's Jump Scare Mansion,"Best game, more better than Sam Pepper's YouTu...",1
2,Spooky's Jump Scare Mansion,"A littly iffy on the controls, but once you kn...",1
3,Spooky's Jump Scare Mansion,"Great game, fun and colorful and all that.A si...",1
4,Spooky's Jump Scare Mansion,Not many games have the cute tag right next to...,1
...,...,...,...
17489,EverQuest II,Arguably the single greatest mmorp that exists...,1
17490,EverQuest II,"An older game, to be sure, but has its own cha...",1
17491,EverQuest II,When I frist started playing Everquest 2 it wa...,1
17492,EverQuest II,cool game. THe only thing that REALLY PISSES M...,1


In [10]:
#Check if any values are empty or null
df_train[df_train.isnull().any(axis=1)]

,title,user_review,user_suggestion


In [11]:
#Tokenize the tweets to get individual words per row.
tkr = RegexpTokenizer("[a-zA-Záéíóúñ']+")
df_train['tokenized'] = df_train['user_review'].apply(lambda row: [t.lower() for t in tkr.tokenize(row)])
df_train

,title,user_review,user_suggestion,tokenized
0,Spooky's Jump Scare Mansion,I'm scared and hearing creepy voices. So I'll...,1,"[i'm, scared, and, hearing, creepy, voices, so..."
1,Spooky's Jump Scare Mansion,"Best game, more better than Sam Pepper's YouTu...",1,"[best, game, more, better, than, sam, pepper's..."
2,Spooky's Jump Scare Mansion,"A littly iffy on the controls, but once you kn...",1,"[a, littly, iffy, on, the, controls, but, once..."
3,Spooky's Jump Scare Mansion,"Great game, fun and colorful and all that.A si...",1,"[great, game, fun, and, colorful, and, all, th..."
4,Spooky's Jump Scare Mansion,Not many games have the cute tag right next to...,1,"[not, many, games, have, the, cute, tag, right..."
...,...,...,...,...
17489,EverQuest II,Arguably the single greatest mmorp that exists...,1,"[arguably, the, single, greatest, mmorp, that,..."
17490,EverQuest II,"An older game, to be sure, but has its own cha...",1,"[an, older, game, to, be, sure, but, has, its,..."
17491,EverQuest II,When I frist started playing Everquest 2 it wa...,1,"[when, i, frist, started, playing, everquest, ..."
17492,EverQuest II,cool game. THe only thing that REALLY PISSES M...,1,"[cool, game, the, only, thing, that, really, p..."


In [12]:
#Drop rows that have more than 150 words
df_train=df_train[df_train["tokenized"].str.len() <= 150]
df_train=df_train[df_train["tokenized"].str.len() > 3]
df_train=df_train[~df_train["user_review"].str.contains(r'[~@#$%^*/\|<>♥¥™_=+◕ヘ░▄▐☉ʖ]')] 
df_train

,title,user_review,user_suggestion,tokenized
0,Spooky's Jump Scare Mansion,I'm scared and hearing creepy voices. So I'll...,1,"[i'm, scared, and, hearing, creepy, voices, so..."
3,Spooky's Jump Scare Mansion,"Great game, fun and colorful and all that.A si...",1,"[great, game, fun, and, colorful, and, all, th..."
4,Spooky's Jump Scare Mansion,Not many games have the cute tag right next to...,1,"[not, many, games, have, the, cute, tag, right..."
6,Spooky's Jump Scare Mansion,Great game. it's a cute little horror game tha...,1,"[great, game, it's, a, cute, little, horror, g..."
9,Spooky's Jump Scare Mansion,This game with its cute little out of the wall...,1,"[this, game, with, its, cute, little, out, of,..."
...,...,...,...,...
17486,EverQuest II,I have always loved this game! I love the idea...,1,"[i, have, always, loved, this, game, i, love, ..."
17487,EverQuest II,After I had played it extensively for four yea...,1,"[after, i, had, played, it, extensively, for, ..."
17488,EverQuest II,this was the first MMORPG i've ever played but...,0,"[this, was, the, first, mmorpg, i've, ever, pl..."
17492,EverQuest II,cool game. THe only thing that REALLY PISSES M...,1,"[cool, game, the, only, thing, that, really, p..."


#Data Preprocessing (Test Dataset)
NOTES: 

**Should we include numbers in the tokenized text?**

**Will different genres effect our model?** 

**There are different game compared to the training dataset**

In [13]:
#Read the review dataset from the training folder
filename_read = "https://raw.githubusercontent.com/JasonTPhillipsJr/CSC215/main/Final%20Project/test.csv"
df_test = pd.read_csv(filename_read, sep=',', header=0)
print("Test data shape: " + str(df_test.shape))
df_test

Test data shape: (8045, 4)


,review_id,title,year,user_review
0,1603,Counter-Strike: Global Offensive,2015.0,"Nice graphics, new maps, weapons and models. B..."
1,1604,Counter-Strike: Global Offensive,2018.0,I would not recommend getting into this at its...
2,1605,Counter-Strike: Global Offensive,2018.0,Edit 11/12/18I have tried playing CS:GO recent...
3,1606,Counter-Strike: Global Offensive,2015.0,The game is great. But the community is the wo...
4,1607,Counter-Strike: Global Offensive,2015.0,I thank TrulyRazor for buying this for me a lo...
...,...,...,...,...
8040,25198,GUNS UP!,2017.0,Early Access ReviewGuns UP!Positive: Good Idea...
8041,25199,GUNS UP!,2018.0,"After 170 hrs, my review. DO NOT SPEND ANY MON..."
8042,25200,GUNS UP!,2018.0,Pros:-Fun GameplayCons: -Micro %$#*ing transac...
8043,25201,GUNS UP!,2018.0,"Actualy saucy, I definetly suggest to players ..."


In [14]:
#Print out the different games used in this review.
#NOTE: Remove the ® symbol from smite and elder scrolls later.
print("Number of titles in the dataset: " + str(df_test['title'].nunique()))
print(df_test['title'].unique())

Number of titles in the dataset: 20
['Counter-Strike: Global Offensive' 'World of Warships' 'Star Trek Online'
 'Paladins®' 'Shadowverse CCG' 'Tree of Savior (English Ver.)'
 'VEGA Conflict' 'Minion Masters' 'The Lord of the Rings Online™'
 'Fishing Planet' 'Crush Crush' 'Dungeon Defenders II'
 'Governor of Poker 3' 'Digimon Masters Online' 'Shakes and Fidget'
 'Champions Online' 'Magic Duels' 'Aura Kingdom' 'H1Z1' 'GUNS UP!']


In [15]:
#Drop unnecessary columns
#NOTE: DROP THE TITLE I THINK???
df_test = df_test.drop(['review_id', 'year'], axis=1)
print("Training data shape: " + str(df_test.shape))
df_test.head()

Training data shape: (8045, 2)


,title,user_review
0,Counter-Strike: Global Offensive,"Nice graphics, new maps, weapons and models. B..."
1,Counter-Strike: Global Offensive,I would not recommend getting into this at its...
2,Counter-Strike: Global Offensive,Edit 11/12/18I have tried playing CS:GO recent...
3,Counter-Strike: Global Offensive,The game is great. But the community is the wo...
4,Counter-Strike: Global Offensive,I thank TrulyRazor for buying this for me a lo...


In [16]:
#Check if any values are empty or null
df_test[df_test.isnull().any(axis=1)]

,title,user_review


In [17]:
#Tokenize the tweets to get individual words per row.
tkr = RegexpTokenizer("[a-zA-Záéíóúñ']+")
df_test['tokenized'] = df_test['user_review'].apply(lambda row: [t.lower() for t in tkr.tokenize(row)])
df_test

,title,user_review,tokenized
0,Counter-Strike: Global Offensive,"Nice graphics, new maps, weapons and models. B...","[nice, graphics, new, maps, weapons, and, mode..."
1,Counter-Strike: Global Offensive,I would not recommend getting into this at its...,"[i, would, not, recommend, getting, into, this..."
2,Counter-Strike: Global Offensive,Edit 11/12/18I have tried playing CS:GO recent...,"[edit, i, have, tried, playing, cs, go, recent..."
3,Counter-Strike: Global Offensive,The game is great. But the community is the wo...,"[the, game, is, great, but, the, community, is..."
4,Counter-Strike: Global Offensive,I thank TrulyRazor for buying this for me a lo...,"[i, thank, trulyrazor, for, buying, this, for,..."
...,...,...,...
8040,GUNS UP!,Early Access ReviewGuns UP!Positive: Good Idea...,"[early, access, reviewguns, up, positive, good..."
8041,GUNS UP!,"After 170 hrs, my review. DO NOT SPEND ANY MON...","[after, hrs, my, review, do, not, spend, any, ..."
8042,GUNS UP!,Pros:-Fun GameplayCons: -Micro %$#*ing transac...,"[pros, fun, gameplaycons, micro, ing, transact..."
8043,GUNS UP!,"Actualy saucy, I definetly suggest to players ...","[actualy, saucy, i, definetly, suggest, to, pl..."


In [18]:
#Drop rows that have more than 150 words
df_test=df_test[df_test["tokenized"].str.len() <= 150]
df_test=df_test[~df_test["user_review"].str.contains(r'[~@#$%^*/\|<>♥¥™_=+◕ヘ░▄▐☉ʖ]')] 
df_test

,title,user_review,tokenized
0,Counter-Strike: Global Offensive,"Nice graphics, new maps, weapons and models. B...","[nice, graphics, new, maps, weapons, and, mode..."
3,Counter-Strike: Global Offensive,The game is great. But the community is the wo...,"[the, game, is, great, but, the, community, is..."
15,Counter-Strike: Global Offensive,I don't know what it was like before it went F...,"[i, don't, know, what, it, was, like, before, ..."
16,Counter-Strike: Global Offensive,"Extremely toxic community, you will be lucky t...","[extremely, toxic, community, you, will, be, l..."
20,Counter-Strike: Global Offensive,This game will give you low trust factor after...,"[this, game, will, give, you, low, trust, fact..."
...,...,...,...
8036,GUNS UP!,Early Access Reviewincredibly straight forward...,"[early, access, reviewincredibly, straight, fo..."
8038,GUNS UP!,Early Access ReviewActive Dev Team.Gets update...,"[early, access, reviewactive, dev, team, gets,..."
8040,GUNS UP!,Early Access ReviewGuns UP!Positive: Good Idea...,"[early, access, reviewguns, up, positive, good..."
8043,GUNS UP!,"Actualy saucy, I definetly suggest to players ...","[actualy, saucy, i, definetly, suggest, to, pl..."


#Word Embeddings (Word2Vec)
Create word embeddings so that we have something to input to our model.

In [ ]:
"""
#This goes after word embedding .-.
from tensorflow.keras.preprocessing import sequence
df_train["tokenized"] = sequence.pad_sequences(df_train["tokenized"], maxlen = 150)
df_train
"""

In [ ]:
df_train

,title,user_review,user_suggestion,tokenized
0,Spooky's Jump Scare Mansion,I'm scared and hearing creepy voices. So I'll...,1,"[i'm, scared, and, hearing, creepy, voices, so..."
3,Spooky's Jump Scare Mansion,"Great game, fun and colorful and all that.A si...",1,"[great, game, fun, and, colorful, and, all, th..."
4,Spooky's Jump Scare Mansion,Not many games have the cute tag right next to...,1,"[not, many, games, have, the, cute, tag, right..."
6,Spooky's Jump Scare Mansion,Great game. it's a cute little horror game tha...,1,"[great, game, it's, a, cute, little, horror, g..."
9,Spooky's Jump Scare Mansion,This game with its cute little out of the wall...,1,"[this, game, with, its, cute, little, out, of,..."
...,...,...,...,...
17486,EverQuest II,I have always loved this game! I love the idea...,1,"[i, have, always, loved, this, game, i, love, ..."
17487,EverQuest II,After I had played it extensively for four yea...,1,"[after, i, had, played, it, extensively, for, ..."
17488,EverQuest II,this was the first MMORPG i've ever played but...,0,"[this, was, the, first, mmorpg, i've, ever, pl..."
17492,EverQuest II,cool game. THe only thing that REALLY PISSES M...,1,"[cool, game, the, only, thing, that, really, p..."


In [ ]:
X_train = df_train["tokenized"]
X_test = df_train["user_suggestion"]
print(f"{X_train.shape} and {X_test.shape}")

(8000,) and (8000,)


KeyError: ignored

#Word Embeddings (Bert)
Context Sensitive embeddings

In [19]:
import torch
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [20]:
sentences = df_train.user_review.values
labels = df_train.user_suggestion.values

In [21]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [22]:
# Print the original sentence.
print(' Original: ', sentences[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  I'm scared and hearing creepy voices.  So I'll pause for a moment and write a review while I wait for my heart beat to return to atleast somewhat calmer times.  This game is adorable and creepy like my happy tree friends but with the graphics sceme of my childhood (but more bubble and 'clean').  Hello 1990's.What charactes there are (that isnot trying to kill me) were likable and a bit odd.  I did do a few noob things though, such as:Oh look a class room full of ghosts from dead children, lets shine my flashlight on them and stand there staring at them..Or, hmm creepy music, I'll turn around and see if I can see what's chasing me.Never before in a game have I been this afraid of finding a locked door.
Tokenized:  ['i', "'", 'm', 'scared', 'and', 'hearing', 'creepy', 'voices', '.', 'so', 'i', "'", 'll', 'pause', 'for', 'a', 'moment', 'and', 'write', 'a', 'review', 'while', 'i', 'wait', 'for', 'my', 'heart', 'beat', 'to', 'return', 'to', 'at', '##lea', '##st', 'somewhat', 'ca

In [23]:
#max_len = 150

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 150,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  I'm scared and hearing creepy voices.  So I'll pause for a moment and write a review while I wait for my heart beat to return to atleast somewhat calmer times.  This game is adorable and creepy like my happy tree friends but with the graphics sceme of my childhood (but more bubble and 'clean').  Hello 1990's.What charactes there are (that isnot trying to kill me) were likable and a bit odd.  I did do a few noob things though, such as:Oh look a class room full of ghosts from dead children, lets shine my flashlight on them and stand there staring at them..Or, hmm creepy music, I'll turn around and see if I can see what's chasing me.Never before in a game have I been this afraid of finding a locked door.
Token IDs: tensor([  101,  1045,  1005,  1049,  6015,  1998,  4994, 17109,  5755,  1012,
         2061,  1045,  1005,  2222,  8724,  2005,  1037,  2617,  1998,  4339,
         1037,  3319,  2096,  1045,  3524,  2005,  2026,  2540,  3786,  2000,
         2709,  2000,  2012, 1973

In [36]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

7,200 training samples
  800 validation samples


In [37]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [38]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [39]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [40]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [33]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [34]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [35]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [42]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A100-SXM4-40GB


In [45]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels).to_tuple()

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels).to_tuple()
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    225.    Elapsed: 0:00:08.
  Batch    80  of    225.    Elapsed: 0:00:16.
  Batch   120  of    225.    Elapsed: 0:00:24.
  Batch   160  of    225.    Elapsed: 0:00:32.
  Batch   200  of    225.    Elapsed: 0:00:40.

  Average training loss: 0.13
  Training epcoh took: 0:00:45

Running Validation...
  Accuracy: 0.91
  Validation Loss: 0.33
  Validation took: 0:00:02

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    225.    Elapsed: 0:00:08.
  Batch    80  of    225.    Elapsed: 0:00:16.
  Batch   120  of    225.    Elapsed: 0:00:24.
  Batch   160  of    225.    Elapsed: 0:00:32.
  Batch   200  of    225.    Elapsed: 0:00:40.

  Average training loss: 0.12
  Training epcoh took: 0:00:45

Running Validation...
  Accuracy: 0.91
  Validation Loss: 0.32
  Validation took: 0:00:02

======== Epoch 3 / 4 ========
Training...
  Batch    40  of    225.    Elapsed: 0:00:08.
  Batch    80  of    225.    Elapsed: 0:00:16.
 